In [5]:
import torch
import librosa

from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

In [6]:
# load pretrained model
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
def wav2vec_inference(wav_file):

    # load audio
    audio_input, sample_rate = librosa.load(wav_file, sr=16000)

    # pad input values and return pt tensor
    input_values = processor(audio_input, sampling_rate=sample_rate, return_tensors="pt").input_values

    #INFERENCE

    # retrieve logits & take argmax
    logits = model(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)

    #transcribe
    transcription = processor.decode(predicted_ids[0])

    return transcription


In [8]:
leminh_recording = 'audio_samples/leminh-wind-09092021.wav'
tatsu_recording = 'audio_samples/tatsu-wind-09092021.wav'

print()
leminh_prediction = wav2vec_inference(leminh_recording)
print(f'Prediction of {os.path.basename(leminh_recording)}: {leminh_prediction}')
print()
tatsu_prediction = wav2vec_inference(tatsu_recording)
print(f'Prediction of {os.path.basename(tatsu_recording)}: {tatsu_prediction}')

NameError: name 'os' is not defined

## The code snippet below is for WER calcutation.

In [ ]:
from jiwer import wer

# the script for the dictation
original_text = "THE NORTH WIND AND THE SUN WERE DISPUTING WHICH WAS THE STRONGER WHEN HE TRAVELLER CAME ALONG WRAPPED IN A WARM CLOAK DAY AGREED THAT THE ONE WHO FIRST SUCCEEDED IN MAKING THE TRAVELR TAKE HIS CLOAK OFF SHOULD BE CONSIDERED STRONGER THAN THE OTHER THEN THE NORTH WIND BLEW AS HARD AS HE COULD BUT THE MORE HE BLEW THE MORE CLOSELY TID THE TRAVELLER FOGT HIS CLOAK AROUND HIM AND AT LAST THE NORTH WIND GAVE UP THE ATTEMPT THEN THE SUN SHUNED OUT WARMLY AND IMMEDIDAELY THE TRAVELLER TOOK OFF HIS CLOAK AND SO THE NORTH WIND WAS OBLIGED TO CONFESS THAT THE SUN WAS THE STRONGER OFF THE TWIAK"

# WER is calculated in percentage.
WER_leminh = wer(original_text, leminh_prediction)
WER_tatsu = wer(original_text, tatsu_prediction)

print(wer)